# notebook from https://github.com/aamamaev/machine-learning/blob/master/parsing/parsing_example.ipynb

In [1]:
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime
from tqdm import tqdm_notebook

from collections import namedtuple
import pickle
from IPython.display import clear_output
from selenium import webdriver
from time import sleep

In [89]:
atm = pd.read_csv('../data/input/test.csv',index_col=0)

In [90]:
atm.head()
to_repair = atm.iloc[np.where(atm.lat.isna())[0],:]
to_repair

,id,atm_group,address,address_rus,lat,long
1,2570.0,3185.5,UL. SERGEJA MAKEEVA MOSKVA,NaN,NaN,NaN
90,6662.0,5478.0,"D. 12, UL. AKADEMICHESKAY IRKUTSK G",NaN,NaN,NaN
99,484.0,1942.0,93 KRASNOARMEYSKAYA SAMARA,NaN,NaN,NaN
137,1469.0,1942.0,20 GUSINOBRODSKOE H/W NOVOSIBIRSK,NaN,NaN,NaN
146,3284.0,8083.0,"YA KOMSOMOLSK,19A Novo-Aleksandro",NaN,NaN,NaN
168,416.0,1942.0,102/4A YAMSKAYA STR TYUMEN,NaN,NaN,NaN
180,5085.0,5478.0,"D. 9, UL. KUZOVATOVSKAYA ULYANOVSK G",NaN,NaN,NaN
185,1013.0,1942.0,34 LENINGRADSKIY AVE MOSCOW,NaN,NaN,NaN
186,6912.0,5478.0,"D. 15, UL. BELORECHENSKAY EKATERINBURG",NaN,NaN,NaN
220,4427.0,8083.0,"LENINSKAY, 37 Pervomayskoye",NaN,NaN,NaN


In [91]:
to_repair['street'] = to_repair.address.apply(lambda x: x[:23].strip())

to_repair['city'] = to_repair.address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

In [116]:
def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "Moscva": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

In [93]:
to_repair.city = to_repair.city.map(transform)

In [112]:
def tmp_(to_repair):
    to_replace = {
         'AV.': ' Проспект ',
         'AVE': ' Проспект ', 
         'PR-KT': ' Проспект ',
         'PR-T': ' Проспект ',
         'PROSPECT': ' Проспект ',
         'STR.': ' улица ',
         ' ST ': ' улица ',
         'UL.' : ' улица ',
         'ULITSA': ' улица ',
         'SHOSSE': ' шоссе ',
         'H/W.': ' шоссе ',
         ', LIT. ': '',
         'D.': '',
         'VLAD. ':'',
         ', KORP. ': ' корпус ',
         ' DZERZHINSK':  ' дзержинского ',
         '.': '. ',
         ',': ', ',
    }

    for i in to_repair.index:
        for key, value in to_replace.items():
            to_repair.at[i,'street'] =  to_repair.at[i,'street'].replace(key, value)
#             print(to_repair.at[i,'street'])
    for i in to_repair.index:
        to_repair.at[i,'street'] = to_repair.at[i,'street'][4:] if to_repair.at[i,'street'][:4] == 'ABB ' else to_repair.at[i,'street']
    
    return to_repair

In [113]:
np.where(atm_new.lat.isna())[0].shape

(56,)

In [114]:
atm_new = tmp_(atm_new)

In [115]:
atm_new.iloc[np.where(atm_new.lat.isna())[0],:]

,id,atm_group,address,address_rus,lat,long,street,city
90,6662.0,5478.0,"D. 12, UL. AKADEMICHESKAY IRKUTSK G",NaN,NaN,NaN,"12, улица AKADEMICHESK",AY IRKUTSK
146,3284.0,8083.0,"YA KOMSOMOLSK,19A Novo-Aleksandro",NaN,NaN,NaN,"YA KOMSOMOLSK, 19A",Novo-Aleksandro
186,6912.0,5478.0,"D. 15, UL. BELORECHENSKAY EKATERINBURG",NaN,NaN,NaN,"15, улица BELORECHENSK",AY EKATERINBURG
237,5687.0,5478.0,"D. 24, PR-KT KRASNYH PART USOLYE-SIBIR",NaN,NaN,NaN,"24, Проспект KRASNYH PA",RT USOLYE-SIBIR
304,8324.0,5478.0,"D. 16A, UL. INTERNACIONAL TAMBOV G",NaN,NaN,NaN,"16A, улица INTERNACION",AL TAMBOV
318,5648.0,5478.0,"D. 88, PR-KT ZOI KOSMODEM AZOV G",NaN,NaN,NaN,"88, Проспект ZOI KOSMOD",EM AZOV
383,5530.0,5478.0,UNKNOWN DZERZHINSK G,NaN,NaN,NaN,UNKNOWN,DZERZHINSK
396,2041.0,3185.5,"ALTUFEVSKOE70,1. MOSKVA",NaN,NaN,NaN,"ALTUFEVSKOE70, 1.",MOSCOW
442,6339.0,5478.0,"D. 22, PR-KT BRAT'EV KORO ORENBURG G",NaN,NaN,NaN,"22, Проспект BRAT'EV KO",RO ORENBURG
451,7232.0,5478.0,"D. 14, UL. STANKOSTROITEL ULYANOVSK G",NaN,NaN,NaN,"14, улица STANKOSTROIT",EL ULYANOVSK


In [117]:
def parsing(atm, columns=['lat', 'long'], start=0):
    driver = webdriver.Firefox()
    for column in columns:
        atm[column] = np.nan
    url = 'https://google.com/maps/search/'
    for i, atmi in tqdm_notebook(list(atm.iterrows())[start:]):
        urli = url + ' '.join([atmi.street, atmi.city]).replace(' ', '+')
        try:
            driver.get(urli)
        except:
            print(i, urli)
            return (i, atm.copy())
        sleep(10)
        try: 
#             element = driver.find_element_by_xpath()
            curr_url = driver.current_url
            coords = curr_url.split('/')[6].split(',')[0:2]
            coords[0] = coords[0][1:]
            lat, long = map(float, coords)
            print(lat, long)
            atm.loc[i, 'lat'] = lat
            atm.loc[i, 'long'] = long
        except:
            pass
    return atm.copy()

In [118]:
a = parsing(atm_new.iloc[np.where(atm_new.lat.isna())[0],:])

52.234381 104.287982
55.980327 37.55472
56.818218 60.5661673
52.7321017 103.6565166
52.7209518 41.4543059
47.100403 39.419665
56.2582775 43.5047387
55.888256 37.5885349
51.7909394 55.1024234
54.3111775 48.4662194
51.7120075 36.1822814
52.3061626 104.2964034
55.329916 86.128696
60.0073124 30.2929606
47.100403 39.419665
60.023126 30.376657
55.9477961 37.4980571
56.1090093 47.3302255
55.329916 86.128696
55.794882 37.7128041
54.5350486 36.2033353
52.0473769 113.4753941
59.9164335 30.3276315
50.5491344 137.0177293
54.9912106 73.3653835
51.513624 45.993583
55.7227612 37.570661
54.9575575 82.9006196
52.7321017 103.6565166
56.0186222 92.9707999
55.1871171 61.3335714
47.100403 39.419665
59.8480206 30.3222348
56.3643965 43.8156216
55.7354501 37.5115228
51.5502887 46.0001406
55.7437855 37.5660974
55.6393866 37.7585944
55.3112084 86.0905395
59.831752 30.505033
56.871016 35.8877731
61.2540035 73.4333099
59.8480206 30.3222348
59.7994578 30.4026531
56.818218 60.5661673
52.7321017 103.6565166
57.62465

In [73]:
tmp = 'https://www.google.com/maps/place/%D0%98%D1%80%D0%BA%D1%83%D1%82%D1%81%D0%BA/@52.2831698,104.2597307,17z/data=!3m1!4b1!4m5!3m4!1s0x5da824bbd21fb7c7:0x7ba8c402f0ad1da7!8m2!3d52.2831698!4d104.2597307'

In [76]:
tmp.split('/')[6].split(',')[0:2]

['@52.2831698', '104.2597307']

In [122]:
atm_new.iloc[np.where(atm_new.lat.isna())[0], [4, 5]] = a.iloc[:, [4, 5]].values

In [123]:
atm_new

,id,atm_group,address,address_rus,lat,long,street,city
1,2570.0,3185.5,UL. SERGEJA MAKEEVA MOSKVA,NaN,55.763128,37.551663,улица SERGEJA MAKEEVA,MOSCOW
90,6662.0,5478.0,"D. 12, UL. AKADEMICHESKAY IRKUTSK G",NaN,52.234381,104.287982,"12, улица AKADEMICHESK",AY IRKUTSK
99,484.0,1942.0,93 KRASNOARMEYSKAYA SAMARA,NaN,53.190591,50.114971,93 KRASNOARMEYSKAYA,SAMARA
137,1469.0,1942.0,20 GUSINOBRODSKOE H/W NOVOSIBIRSK,NaN,55.030199,82.920430,20 GUSINOBRODSKOE H/W,NOVOSIBIRSK
146,3284.0,8083.0,"YA KOMSOMOLSK,19A Novo-Aleksandro",NaN,55.980327,37.554720,"YA KOMSOMOLSK, 19A",Novo-Aleksandro
168,416.0,1942.0,102/4A YAMSKAYA STR TYUMEN,NaN,55.774132,37.592186,102/4A YAMSKAYA STR,TYUMEN
180,5085.0,5478.0,"D. 9, UL. KUZOVATOVSKAYA ULYANOVSK G",NaN,54.278987,48.301146,"9, улица KUZOVATOVSKAY",A ULYANOVSK
185,1013.0,1942.0,34 LENINGRADSKIY AVE MOSCOW,NaN,55.785761,37.568821,34 LENINGRADSKIY Проспект,MOSCOW
186,6912.0,5478.0,"D. 15, UL. BELORECHENSKAY EKATERINBURG",NaN,56.818218,60.566167,"15, улица BELORECHENSK",AY EKATERINBURG
220,4427.0,8083.0,"LENINSKAY, 37 Pervomayskoye",NaN,55.530281,37.156063,"LENINSKAY, 37",Pervomayskoye


In [124]:
atm.iloc[np.where(atm.lat.isna())[0], [4, 5]] = atm_new.loc[:, ['lat', 'long']].values

In [125]:
atm.to_csv('../data/working/test_more_addr.csv')